# Phase 4: Data Validation & Integrity Audit

**Project**: Student Stress Risk Prediction with Explainable AI  
**Lead**: Data Integrity Officer & Validation Authority

## Objectives
- Validate schema integrity.
- Perform range and plausibility checks.
- Detect compositional constraints (Total Daily Hours).
- Audit multivariate outliers using IQR method.

In [ ]:
import pandas as pd
import numpy as np
import json
import os

df = pd.read_csv('../data/raw.csv')
print(f"Dataset Loaded: {df.shape}")

## 1. Schema Validation

In [ ]:
schema_observed = df.dtypes
print("Observed Schema:")
print(schema_observed)

## 2. Missingness Analysis

In [ ]:
missing = df.isnull().sum()
print("Missing Values per Feature:")
print(missing)

## 3. Range & Plausibility Checks
Checking min/max bounds for all numerical features.

In [ ]:
num_cols = ['Study_Hours_Per_Day', 'Extracurricular_Hours_Per_Day', 'Sleep_Hours_Per_Day', 'Social_Hours_Per_Day', 'Physical_Activity_Hours_Per_Day', 'GPA']
display(df[num_cols].describe().T[['min', 'max']])

## 4. Cross-Feature Constraint: The 24-Hour Law
Validating if the sum of lifestyle activities exceeds or equals the 24-hour daily budget.

In [ ]:
hour_cols = ['Study_Hours_Per_Day', 'Extracurricular_Hours_Per_Day', 'Sleep_Hours_Per_Day', 'Social_Hours_Per_Day', 'Physical_Activity_Hours_Per_Day']
df['Total_Hours'] = df[hour_cols].sum(axis=1)

print(f"Total Hours Statistics:")
print(f"Mean: {df['Total_Hours'].mean():.2f}")
print(f"Min:  {df['Total_Hours'].min():.2f}")
print(f"Max:  {df['Total_Hours'].max():.2f}")

violations = df[np.abs(df['Total_Hours'] - 24.0) > 0.01]
print(f"\nNumber of 24-hour constraint violations: {len(violations)}")

## 5. Outlier Detection (Interquartile Range Method)
Identifying statistical anomalies in univariate distributions.

In [ ]:
print("--- Outlier Detection ---")
for col in num_cols:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"{col}: {len(outliers)} outliers (Range: {lower_bound:.2f} to {upper_bound:.2f})")

## 6. Audit Verdict
**Verdict: VALID & COMPOSITIONAL**

- **Integrity**: The dataset is internally consistent with zero missing values.
- **Constraint**: Perfect linear dependency detected (Sum of hours = 24.0). One feature must be omitted during modelling to prevent multicollinearity.
- **Outliers**: Extreme values in `Physical_Activity` are logically balanced by other features to maintain the 24-hour daily budget.